<a href="https://colab.research.google.com/github/bethbirhanu/MSDS696_S70_Data-Science-Practicum-II/blob/main/Part_2_Week4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:

import pandas as pd
import os

In [3]:
import pandas as pd

# Load the cleaned 2007-2020 dataset from the CSV file
df_2007_2020_clean = pd.read_csv('/content/drive/My Drive/df_2007_2020_clean.csv')

# Load the cleaned 2021-2022 dataset from the CSV file
df_2021_2022_clean = pd.read_csv('/content/drive/My Drive/df_2021_2022_clean.csv')

In [4]:


# Select columns with non-numeric data types
categorical_cols = df_2007_2020_clean.select_dtypes(include=['object', 'category']).columns


for col in df_2007_2020_clean.select_dtypes(include=['int64', 'float64']).columns:
    if len(df_2007_2020_clean[col].unique()) < 10:  # the threshold can be adjusted
        categorical_cols = categorical_cols.append(pd.Index([col]))

# Print out the categorical columns
print(categorical_cols)


Index(['SYSTEM', 'DATE', 'SEVERITY', 'ROAD_DESC', 'CONTOUR', 'CONDITION',
       'LIGHTING', 'WEATHER', 'LIMIT1', 'RUCODE', 'MHE', 'VEHICLE_1',
       'DRIVER_1', 'FACTOR_1', 'VEH_MOVE_1', 'SEX_1', 'CITY', 'COUNTY',
       'REGION'],
      dtype='object')


In [5]:
# Select columns with non-numeric data types
categorical_cols1 = df_2021_2022_clean.select_dtypes(include=['object', 'category']).columns

for col in df_2021_2022_clean.select_dtypes(include=['int64', 'float64']).columns:
    if len(df_2021_2022_clean[col].unique()) < 10:  # the threshold can be adjusted
        categorical_cols1 = categorical_cols1.append(pd.Index([col]))

# Print out the categorical columns
print(categorical_cols1)

Index(['SYSTEM', 'DATE', 'SEVERITY', 'ROAD_DESC', 'CONTOUR', 'CONDITION',
       'LIGHTING', 'WEATHER', 'LIMIT1', 'RUCODE', 'MHE', 'VEHICLE_1',
       'DRIVER_1', 'FACTOR_1', 'VEH_MOVE_1', 'SEX_1', 'CITY', 'COUNTY',
       'INJURY 02', 'REGION'],
      dtype='object')


#### using one-hot encoding 'get dummy' function changing catagorical variable to numeric

In [6]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder


# Combine datasets to ensure consistent encoding
combined_df = pd.concat([df_2007_2020_clean, df_2021_2022_clean], sort=False)

# One-hot encoding
one_hot_cols = ['SYSTEM', 'ROAD_DESC', 'CONDITION', 'LIGHTING', 'WEATHER', 'CONTOUR', 'RUCODE', 'MHE', 'VEHICLE_1', 'DRIVER_1', 'FACTOR_1', 'VEH_MOVE_1', 'CITY', 'COUNTY']
combined_df_encoded = pd.get_dummies(combined_df, columns=one_hot_cols)

# Label encoding for any ordinal categories or high cardinality nominal categories
label_encoder = LabelEncoder()
label_cols = ['SEX_1', 'REGION']
for col in label_cols:
    combined_df_encoded[col] = label_encoder.fit_transform(combined_df_encoded[col].astype(str))

# Split the combined dataset back into training and testing sets
df_2007_2020_encoded = combined_df_encoded[:len(df_2007_2020_clean)]
df_2021_2022_encoded = combined_df_encoded[len(df_2007_2020_clean):]




In [7]:
# Use pandas get_dummies for one-hot encoding
df_2007_2020_encoded = pd.get_dummies(df_2007_2020_encoded, columns=['REGION'])
df_2021_2022_encoded = pd.get_dummies(df_2021_2022_encoded, columns=['REGION'])

# Ensure both dataframes have the same set of columns after encoding,
missing_cols = set(df_2007_2020_encoded.columns) - set(df_2021_2022_encoded.columns)
for c in missing_cols:
    df_2021_2022_encoded[c] = 0

missing_cols = set(df_2021_2022_encoded.columns) - set(df_2007_2020_encoded.columns)
for c in missing_cols:
    df_2007_2020_encoded[c] = 0

# Align the order of columns in both dataframes
df_2007_2020_encoded, df_2021_2022_encoded = df_2007_2020_encoded.align(df_2021_2022_encoded, axis=1)


In [10]:
# Assuming df_2007_2020_encoded is your dataset
X_train = df_2007_2020_encoded.drop('SEVERITY', axis=1)  # Drop the target variable to get the features
y_train = df_2007_2020_encoded['SEVERITY']  # Target variable



In [11]:
# Convert 'DATE' to datetime if it's in string format and extract year, month, and day
if 'DATE' in X_train.columns:
    X_train['DATE'] = pd.to_datetime(X_train['DATE'])
    X_train['YEAR'] = X_train['DATE'].dt.year
    X_train['MONTH'] = X_train['DATE'].dt.month
    X_train['DAY'] = X_train['DATE'].dt.day
    X_train = X_train.drop('DATE', axis=1)  # Drop the original 'DATE' column if no longer needed

# One-hot encode categorical variables if they exist
categorical_columns = X_train.select_dtypes(include=['object', 'category']).columns
X_train = pd.get_dummies(X_train, columns=categorical_columns)

# Now let's retry the feature selection
from sklearn.feature_selection import SelectKBest, f_classif

# Define the number of features you want to keep
k = 10  # or adjust based on your preference

selector = SelectKBest(score_func=f_classif, k=k)
X_train_selected = selector.fit_transform(X_train, y_train)

selected_features = X_train.columns[selector.get_support()]
print(f"Selected features using ANOVA F-test: {selected_features.tolist()}")



/usr/local/lib/python3.10/dist-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [56] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


Selected features using ANOVA F-test: ['INJURY 00', 'INJURY 01', 'INJURY 02', 'INJURY 03', 'MHE_BICYCLE', 'MHE_OVERTURNING', 'MHE_PEDESTRIAN', 'MHE_SIDESWIPE (SAME DIRECTION)', 'MHE_UNKNOWN', 'VEHICLE_1_MOTORCYCLE']


### Modeling Phase

### Random Forest Classifier

#### Step 1: Prepare the Data
I need to ensure that both the training and testing datasets contain only the features selected during the feature selection phase. I will also extract the target variable SEVERITY from both datasets.

In [12]:

# Extract the selected features from both datasets
selected_features = ['INJURY 00', 'INJURY 01', 'INJURY 02', 'INJURY 03', 'MHE_BICYCLE', 'MHE_OVERTURNING', 'MHE_PEDESTRIAN', 'MHE_SIDESWIPE (SAME DIRECTION)', 'MHE_UNKNOWN', 'VEHICLE_1_MOTORCYCLE']

X_train = df_2007_2020_encoded[selected_features]
y_train = df_2007_2020_encoded['SEVERITY']

X_test = df_2021_2022_encoded[selected_features]
y_test = df_2021_2022_encoded['SEVERITY']


####Step 2: Train the Model
I will train a Random Forest Classifier using the training data.

In [13]:
from sklearn.ensemble import RandomForestClassifier

# Initialize the Random Forest model
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model
rf_classifier.fit(X_train, y_train)


RandomForestClassifier(random_state=42)

####Step 3: Evaluate the Model
After training the model, I will evaluate its performance on the testing data to see how well it generalizes to new, unseen data.

In [14]:
from sklearn.metrics import classification_report, accuracy_score

# Predict on the testing set
y_pred = rf_classifier.predict(X_test)

# Evaluate the model
print("Classification Report:")
print(classification_report(y_test, y_pred))
print("Accuracy Score:")
print(accuracy_score(y_test, y_pred))


Classification Report:
              precision    recall  f1-score   support

         FAT       0.66      0.32      0.43       575
         INJ       0.99      1.00      1.00     36169
         PDO       1.00      1.00      1.00    103275

    accuracy                           1.00    140019
   macro avg       0.88      0.77      0.81    140019
weighted avg       1.00      1.00      1.00    140019

Accuracy Score:
0.9965219005992044



The model's performance metrics indicate a high overall accuracy score of approximately 99.65%. However, when we look closer at the precision, recall, and f1-score for each class, there's a noticeable difference:

FAT (Fatalities): This class has much lower precision and recall compared to the other classes, indicating the model struggles to correctly predict fatal accidents.
INJ (Injuries) and PDO (Property Damage Only): The model performs exceptionally well in predicting these classes, as indicated by the near-perfect scores.

####1. Addressing Class Imbalance
Class imbalance can significantly affect model performance, particularly for minority classes. I can use techniques like SMOTE (Synthetic Minority Over-sampling Technique) to balance the dataset.

In [15]:
from imblearn.over_sampling import SMOTE

# Initialize SMOTE
smote = SMOTE()

# Resample the training data
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Check the balance
print(y_train_resampled.value_counts())


SEVERITY
PDO    768981
INJ    768981
FAT    768981
Name: count, dtype: int64


It looks like I've successfully balanced the classes using an oversampling technique such as SMOTE, resulting in an equal number of instances for the PDO, INJ, and FAT categories in my training data. This is a significant step in addressing class imbalance and should help improve the model's performance, especially for the minority classes.

####2. Hyperparameter Tuning
I can use grid search or random search to find the optimal hyperparameters for the Random Forest model.

In [17]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier

# Define the parameter distributions
param_distributions = {
    'n_estimators': [100, 150, 200, 250, 300],
    'max_depth': [10, 15, 20, 25, None],
    'min_samples_split': [2, 4, 6, 8],
    'min_samples_leaf': [1, 2, 3, 4]
}

# Initialize the randomized search
random_search = RandomizedSearchCV(
    estimator=RandomForestClassifier(random_state=42),
    param_distributions=param_distributions,
    n_iter=3,
    cv=3,
    random_state=42,
    scoring='f1_macro',
    n_jobs=-1   # Use all available cores to speed up the process
)

# Fit the randomized search to the resampled data
random_search.fit(X_train_resampled, y_train_resampled)

# Get the best estimator and parameters
best_rf = random_search.best_estimator_
print("Best parameters found:", random_search.best_params_)



Best parameters found: {'n_estimators': 100, 'min_samples_split': 6, 'min_samples_leaf': 2, 'max_depth': 25}


####Model Re-evaluation
After applying the above strategies, we should re-evaluate the model performance on the testing set

In [18]:
# Predict on the testing set using the best model from hyperparameter tuning
y_pred = best_rf.predict(X_test)

# Evaluate the model
print("Revised Classification Report:")
print(classification_report(y_test, y_pred))


Revised Classification Report:
              precision    recall  f1-score   support

         FAT       0.08      0.73      0.14       575
         INJ       1.00      0.93      0.96     36169
         PDO       1.00      0.98      0.99    103275

    accuracy                           0.96    140019
   macro avg       0.69      0.88      0.70    140019
weighted avg       1.00      0.96      0.98    140019



### 1. Logistic Regression

In [19]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Initialize the model
log_reg = LogisticRegression(max_iter=1000)

# Fit the model
log_reg.fit(X_train_resampled, y_train_resampled)

# Predict on the test set
y_pred_log_reg = log_reg.predict(X_test)

# Evaluate the model
print("Logistic Regression Classification Report:")
print(classification_report(y_test, y_pred_log_reg))


Logistic Regression Classification Report:
              precision    recall  f1-score   support

         FAT       0.05      0.73      0.09       575
         INJ       1.00      0.83      0.91     36169
         PDO       1.00      0.98      0.99    103275

    accuracy                           0.94    140019
   macro avg       0.68      0.85      0.66    140019
weighted avg       0.99      0.94      0.96    140019



####2. Support Vector Machine (SVM)
Effective in high-dimensional spaces and best suited for binary classification tasks.

In [ ]:
from sklearn.svm import SVC

# Initialize the model (using the linear kernel for faster computation)
svm = SVC(kernel='linear')

# Fit the model
svm.fit(X_train_resampled, y_train_resampled)

# Predict on the test set
y_pred_svm = svm.predict(X_test)

# Evaluate the model
print("SVM Classification Report:")
print(classification_report(y_test, y_pred_svm))
